## Get all Symbol Exchange

In [2]:
import sys
from pathlib import Path

import ccxt
import pandas as pd


# Add the parent directory to the system path
parent_dir = Path().resolve().parent
sys.path.append(str(parent_dir))
from utils.general import get_top_symbol_by_volume

In [3]:
def get_top_symbol_by_volume(exchange_id, pair_filter, top_n=100):
    """
    Fetch the top N cryptocurrencies traded against USDT based on 24h volume.

    Args:
        exchange_id (str): The exchange ID (e.g., "binance", "bitget").
        top_n (int): The number of top cryptocurrencies to fetch (default is 100).

    Returns:
        pd.DataFrame: A DataFrame containing the top N USDT pairs by 24h volume.
    """
    
    try:
        # Initialize the exchange
        # Fetch all tickers
        exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})
        tickers = exchange.fetch_tickers()
        
        # Extract relevant data for USDT pairs
        data = []

        for symbol, ticker in tickers.items():
            if symbol.endswith(pair_filter):  # Filter for USDT pairs
                data.append({
                    "symbol": symbol,
                    "volume_24h": ticker.get("quoteVolume", 0),  # 24h trading volume in USDT
                    "price": ticker.get("last", 0),  # Last traded price
                })
        
        # Convert to DataFrame and sort by volume
        df = pd.DataFrame(data)
        df = df.sort_values(by="volume_24h", ascending=False).head(top_n)
        
        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

In [4]:
def get_exchange_symbols(exchange_id):
    """
    Fetch all trading symbols for a given exchange.

    Args:
        exchange_id (str): The exchange ID (e.g., "binance", "bitget").

    Returns:
        list: A list of trading pair symbols (e.g., ["BTC/USDT", "ETH/USDT"]).
    """
    try:
        # Initialize the exchange
        exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})
        
        # Load markets
        markets = exchange.load_markets()
        
        # Extract symbols
        symbols = list(markets.keys())
        return symbols

    except Exception as e:
        print(f"Error fetching symbols for exchange {exchange_id}: {e}")
        return []

In [5]:
exchange_id = "binance"  # Replace with your desired exchange ID
symbols = get_exchange_symbols(exchange_id)

print(f"Total symbols on {exchange_id}: {len(symbols)}")
print("Sample symbols:", symbols[:10])  # Display the first 10 symbols

exchange_id = "bitget"  # Replace with your desired exchange ID
symbols = get_exchange_symbols(exchange_id)

print(f"Total symbols on {exchange_id}: {len(symbols)}")
print("Sample symbols:", symbols[:10])  # Display the first 10 symbols

Total symbols on binance: 3380
Sample symbols: ['ETH/BTC', 'LTC/BTC', 'BNB/BTC', 'NEO/BTC', 'QTUM/ETH', 'EOS/ETH', 'SNT/ETH', 'BNT/ETH', 'BCC/BTC', 'GAS/BTC']
Total symbols on bitget: 1259
Sample symbols: ['CATCH/USDT', 'LUMIA/USDT', 'GOAT/USDT', 'ENA/EUR', 'TON/EUR', 'SXRP/SUSDT', 'SHIB/EUR', 'WLD/EUR', 'DBR/USDT', 'OMNIA/USDT']


In [8]:
def get_exchange_timeframes(exchange_id):
    """
    Get the list of available timeframes for a given exchange.

    Args:
        exchange_id (str): The exchange ID (e.g., "binance", "bitget").

    Returns:
        list: A list of available timeframes for the exchange.
    """
    try:
        # Initialize the exchange
        exchange = getattr(ccxt, exchange_id)({'enableRateLimit': True})

        # Check if the exchange supports timeframes
        if hasattr(exchange, 'timeframes') and exchange.timeframes:
            return list(exchange.timeframes.keys())
        else:
            return []
    except Exception as e:
        print(f"Error fetching timeframes for {exchange_id}: {e}")
        return []

# Example Usage
exchange_id = "bitget"  # Replace with your desired exchange ID
timeframes = get_exchange_timeframes(exchange_id)
print(f"Available timeframes for {exchange_id}: {timeframes}")

Available timeframes for bitget: ['1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '12h', '1d', '3d', '1w', '1M']


In [14]:
from utils.general import get_top_symbol_by_volume
MARKET_TYPE = "future"
exchange = getattr(ccxt, exchange_id)({
       
        'options': {'defaultType': MARKET_TYPE},
    })
df_symbols = get_top_symbol_by_volume(exchange=exchange, pair_filter="/USDT:USDT", top_n=100)
df_symbols

,symbol,volume_24h,price
0,BTC/USDT:USDT,1.608615e+10,98899.900000
2,XRP/USDT:USDT,9.565338e+09,3.086600
1,ETH/USDT:USDT,6.092771e+09,3320.150000
12,SOL/USDT:USDT,3.670947e+08,199.933000
11,DOGE/USDT:USDT,2.250878e+08,0.370150
...,...,...,...
333,CHILLGUY/USDT:USDT,6.454721e+06,0.121400
235,CLOUD/USDT:USDT,6.346329e+06,0.152300
290,KMNO/USDT:USDT,6.331285e+06,0.119360
340,MVL/USDT:USDT,6.175370e+06,0.004797
